In [1]:
import numpy as np # linear algebra
import json
from matplotlib import pyplot as plt
from skimage import color
from skimage.feature import hog
from sklearn import svm
from sklearn.metrics import classification_report,accuracy_score

from skimage.io import imread
from skimage.transform import resize
from skimage.feature import hog
from skimage import exposure
import matplotlib.pyplot as plt
import get_lodging_scores
import cv2  # importing cv
import imutils
import glob

#We need these in this file:
import load_read_name_extractor as lrne
import os
import pickle
#For scorer
from sklearn.metrics import make_scorer
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import make_classification

In [2]:
def it_name_extract_labels_from_img(names):
    plots = []
    temp = []
    for i in range(len(names)):
        name_t = os.path.basename(names[i])
        #name_t = name_t.replace(".tif", "")
        for j in range(1,len(name_t)):
            if (names[i][-j] == "_"):
                temp.append(name_t[-(j-1):len(name_t)].replace(".jpeg", ""))
                break
        temp.append(name_t.replace(temp[0], "").replace("_.jpeg", ""))
        plots.append(temp)
        temp = []
    return plots

In [3]:
features_load = np.load('features/hog_features_hog_cells_8x8_block_2x2.npy', allow_pickle=True)
names_load = np.load('features/img_names_hog_cells_8x8_block_2x2.npy', allow_pickle=True)
print(len(features_load[0]))
print(len(names_load))

37332
4983


In [4]:
labels = get_lodging_scores.get_labels(5)
#[ROI, Visual SCORE, FLIGHTFolder, BIN]
print(features_load[0])
ht_mean = features_load[0].mean(axis=0)
ht_mean

print(labels[0])

[0.25384534 0.13523542 0.0568279  ... 0.08940607 0.16619457 0.06113594]
(61, 50, '160607_Min', 10.0)


In [5]:
names = it_name_extract_labels_from_img(names_load)

In [6]:
def match_pic_label(features, labels, names):
    n = len(labels)
    m = len(names)
    data_frame = []
    for i in range(n):
        for j in range(m):
            if (str(labels[i][0]) == str(names[j][0]) and labels[i][2] == names[j][1]):
                data_frame.append(np.hstack((features[j], labels[i][3])))
    return data_frame

In [7]:
clf = svm.SVC(kernel="rbf", class_weight="balanced")
features_aar = np.array(features_load)
data_frame = match_pic_label(features_load, labels, names)
print(data_frame[0])
np.random.shuffle(data_frame)
data_frame = np.array(data_frame)


[ 0.20380611  0.08864094  0.02353812 ...  0.30848572  0.30848572
 10.        ]


In [8]:
percentage = 80
partition = int(len(features_aar)*percentage/100)

In [9]:
x_train, x_test = data_frame[:partition,:-1],  data_frame[partition:,:-1]
y_train, y_test = data_frame[:partition,-1:].ravel() , data_frame[partition:,-1:].ravel()
print(y_train)

clf.fit(x_train,y_train)

[ 1.  1.  1. ... 13. 13.  8.]


SVC(class_weight='balanced')

In [10]:
y_pred = clf.predict(x_test)

In [11]:
#filename = 'SVM_linear_20bins_cells_8x8_block_2x2_balanced_.sav'
#pickle.dump(clf, open(filename, 'wb'))

In [12]:
#loaded_model = pickle.load(open(filename, 'rb'))
#result = loaded_model.score(X_test, Y_test)

In [13]:
print("Accuracy: "+str(accuracy_score(y_test, y_pred)))
print('\n')
print(classification_report(y_test, y_pred))

Accuracy: 0.4142512077294686


              precision    recall  f1-score   support

         1.0       0.59      0.76      0.66       368
         2.0       0.11      0.26      0.15        54
         3.0       0.11      0.10      0.11        48
         4.0       0.00      0.00      0.00        35
         5.0       0.00      0.00      0.00        35
         6.0       0.14      0.09      0.11        32
         7.0       0.00      0.00      0.00        20
         8.0       1.00      0.07      0.12        30
         9.0       0.00      0.00      0.00        18
        10.0       0.00      0.00      0.00        33
        11.0       1.00      0.06      0.12        16
        12.0       1.00      0.04      0.07        26
        13.0       0.15      0.19      0.17        21
        14.0       0.15      0.21      0.17        33
        15.0       0.36      0.40      0.38        35
        16.0       0.44      0.55      0.49        22
        17.0       0.00      0.00      0.00       

c:\Users\marti\miniconda3\envs\imgp2\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\marti\miniconda3\envs\imgp2\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\marti\miniconda3\envs\imgp2\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [19]:
# Define cost matrix
num_classes = 20
cost_matrix = [[0]*num_classes for i in range(num_classes)]

for i in range(num_classes):
    for j in range(num_classes):
        cost_matrix[i][j] = abs(i-j)
cost_matrix

[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
 [1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18],
 [2, 1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17],
 [3, 2, 1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16],
 [4, 3, 2, 1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
 [5, 4, 3, 2, 1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14],
 [6, 5, 4, 3, 2, 1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13],
 [7, 6, 5, 4, 3, 2, 1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
 [8, 7, 6, 5, 4, 3, 2, 1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11],
 [9, 8, 7, 6, 5, 4, 3, 2, 1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
 [10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0, 1, 2, 3, 4, 5, 6, 7, 8],
 [12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0, 1, 2, 3, 4, 5, 6, 7],
 [13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0, 1, 2, 3, 4, 5, 6],
 [14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0, 1

In [14]:
def custom_scorer(y_true, y_pred):
    """
    Calculates the cost of classifying class i as class j.
    :param y_true: true labels
    :param y_pred: predicted labels
    :return: total cost
    """
    cost = 0
    for i in range(len(y_true)):
        cost += cost_matrix[int(y_true[i])][int(y_pred[i])]
    return cost

# Create scorer
scorer = make_scorer(custom_scorer(y_test,y_pred), greater_is_better=False)

In [17]:
np.logspace(-3, 3, 6)

array([1.00000000e-03, 1.58489319e-02, 2.51188643e-01, 3.98107171e+00,
       6.30957344e+01, 1.00000000e+03])

In [15]:
# Define SVM model
svm = clf

# Define parameter grid for grid search
param_grid = {'C': np.logspace(-3, 3, 6)}

# Perform grid search using custom scorer
grid_search = GridSearchCV(svm, param_grid, scoring=scorer)
grid_search.fit(x_train,y_train)

# Print best parameters and score
print(grid_search.best_params_)
print(grid_search.best_score_)

c:\Users\marti\miniconda3\envs\imgp2\lib\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
c:\Users\marti\miniconda3\envs\imgp2\lib\site-packages\sklearn\model_selection\_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\marti\miniconda3\envs\imgp2\lib\site-packages\sklearn\model_selection\_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "c:\Users\marti\miniconda3\envs\imgp2\lib\site-packages\sklearn\metrics\_scorer.py", line 234, in __call__
    return self._score(
  File "c:\Users\marti\miniconda3\envs\imgp2\lib\site-packages\sklearn\metrics\_scorer.py", line 282, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
TypeError: 'int' object is not callable

  warn

{'C': 0.001}
nan


In [16]:
y_pred = grid_search.predict(x_test)
print("Accuracy: "+str(accuracy_score(y_test, y_pred)))
print('\n')
print(classification_report(y_test, y_pred))

Accuracy: 0.021739130434782608


              precision    recall  f1-score   support

         1.0       0.00      0.00      0.00       368
         2.0       0.00      0.00      0.00        54
         3.0       0.00      0.00      0.00        48
         4.0       0.00      0.00      0.00        35
         5.0       0.00      0.00      0.00        35
         6.0       0.00      0.00      0.00        32
         7.0       0.00      0.00      0.00        20
         8.0       0.00      0.00      0.00        30
         9.0       0.02      1.00      0.04        18
        10.0       0.00      0.00      0.00        33
        11.0       0.00      0.00      0.00        16
        12.0       0.00      0.00      0.00        26
        13.0       0.00      0.00      0.00        21
        14.0       0.00      0.00      0.00        33
        15.0       0.00      0.00      0.00        35
        16.0       0.00      0.00      0.00        22
        17.0       0.00      0.00      0.00     

c:\Users\marti\miniconda3\envs\imgp2\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\marti\miniconda3\envs\imgp2\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\marti\miniconda3\envs\imgp2\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
